<a href="https://colab.research.google.com/github/schaam/ESP32Cam-MQTT/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from keras import backend
# force channels-first ordering
backend.set_image_data_format('channels_last')

In [ ]:
!ls
%cd '/content/drive/My Drive/models/'

drive  sample_data
/content/drive/My Drive/models


For Preprocessing images, and maps, and to postprocess the predictions before saving them </br>

In [ ]:
import numpy as np
import scipy.ndimage
import cv2
b_s=1
def preprocess_maps(paths):
    ims = np.zeros((len(paths), 480, 640))
    padded_map = np.zeros((480, 640), dtype=np.uint8)
    for i, path in enumerate(paths):
        original_map = cv2.imread(path, 0)
        padded_map = original_map #padding(original_map, 480, 640, 1)
        ims[i] = padded_map.astype(np.float32)
        ims[i] /= 255.0
    ims=np.expand_dims(ims,axis=3)
    #ims = ims.transpose((0, 3, 1, 2))
    return ims
oim=np.zeros((b_s, 480, 640, 3))
def preprocess_images(paths):
    ims = np.zeros((len(paths), 480, 640,3))
    #print(paths[0])
    for i, path in enumerate(paths):
        original_image = cv2.imread(path,1)
        #print(original_image.shape)
        oim[i]=original_image
        #print(original_image)
        #if original_image is NULL:
        #  raise NotImplementedError
        #cv2.imshow('Image',original_image)
        #padded_image = padding(original_image, 480, 640, 3)
        ims[i] = oim[i]  #was originally padded_image

    ims[:, :, :, 0] -= 103.939
    ims[:, :, :, 1] -= 116.779
    ims[:, :, :, 2] -= 123.68
    #ims = ims.transpose((0, 3, 1, 2))
    #ims=np.expand_dims(ims,axis=1)
    #print(ims.shape)
    #print('image preprocessed')
    return ims

def postprocess_predictions(pred, shape_r, shape_c):
    predictions_shape = pred.shape
    rows_rate = shape_r / predictions_shape[0]
    cols_rate = shape_c / predictions_shape[1]

    pred = pred / np.max(pred) * 255

    #if rows_rate > cols_rate:
    #    new_cols = (predictions_shape[1] * shape_r) // predictions_shape[0]
    #    pred = cv2.resize(pred, (new_cols, shape_r))
    #    img = pred[:, ((pred.shape[1] - shape_c) // 2):((pred.shape[1] - shape_c) // 2 + shape_c)]
    #else:
    #    new_rows = (predictions_shape[0] * shape_c) // predictions_shape[1]
    #    pred = cv2.resize(pred, (shape_c, new_rows))
    #    img = pred[((pred.shape[0] - shape_r) // 2):((pred.shape[0] - shape_r) // 2 + shape_r), :]
    img = pred[:,:]
    min=np.min(img)
    max=np.max(img)

    for i in range(0,480):
      for j in range(0,640):
        if 30<img[i,j]<140:
          img[i,j]=(img[i,j]-min)/(max-min)*255
        #elif img[i,j]<30:
        #  img[i,j]=img[i,j]/2
        #else:
        #  img[i,j]=np.minimum(img[i,j]*1.25,255)
    img = scipy.ndimage.filters.gaussian_filter(img, sigma=7)
    #img = ((img-np.min(img)) / (np.max(img)-np.min(img))) * 255

    print('postprediction generated')

    return img




The generator for feeding the input images and actual maps to the model

In [ ]:
b_s = 1
imgs_train_path='/content/drive/My Drive/dataset/images/tr/train/'
maps_train_path='/content/drive/My Drive/dataset/maps/tr/train/'
imgs_val_path='/content/drive/My Drive/dataset/images/vali/val/'
maps_val_path='/content/drive/My Drive/dataset/maps/vali/val/'
def generator(b_s=b_s, phase_gen='train'):
  if phase_gen == 'train':
        images = [imgs_train_path + f for f in os.listdir(imgs_train_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        #print(os.listdir(imgs_train_path))
        maps = [maps_train_path + f for f in os.listdir(maps_train_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        #fixs = [fixs_train_path + f for f in os.listdir(fixs_train_path) if f.endswith('.mat')]
  elif phase_gen == 'val':
        images = [imgs_val_path + f for f in os.listdir(imgs_val_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        maps = [maps_val_path + f for f in os.listdir(maps_val_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        #fixs = [fixs_val_path + f for f in os.listdir(fixs_val_path) if f.endswith('.mat')]
  else:
        raise NotImplementedError

  images.sort()
  maps.sort()
  print(len(images))
  #fixs.sort()
  #gaussian = np.zeros((b_s, nb_gaussian, shape_r_gt, shape_c_gt)
  counter = 0
  while True:
        Y = preprocess_maps(maps[counter:counter+b_s])
        #Y_fix = preprocess_fixmaps(fixs[counter:counter + b_s], shape_r_out, shape_c_out)
        yield preprocess_images(images[counter:counter + b_s]), [Y,Y]
        #print("batch output done")
        counter = (counter + b_s) % (len(images))

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import keras.layers as layers
import keras.utils
#from keras.layers import merge, Input, Activation
#rom keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
#from keras.layers.convolutional import AtrousConvolution2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.utils.data_utils import get_file
import os
import warnings

#from . import get_submodules_from_kwargs
#from . import imagenet_utils
#from .imagenet_utils import decode_predictions
#from .imagenet_utils import _obtain_input_shape
from keras import backend as K
#preprocess_input = imagenet_utils.preprocess_input
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
bn_axis=3
#backend = None
#layers = None
#models = None
#keras_utils = None

def identity_block(input_tensor, kernel_size, filters, stage, block, d=0):
    filters1, filters2, filters3 = filters
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    conv_name_base = 'res' + str(stage) + block + '_branch'
    if d==0:
      dilation=(1,1)
    else:
      dilation=(2,2)
    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',dilation_rate=dilation,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size,filters,stage,block,strides=(2, 2),d=0):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    if d==0:
      dilation=(1,1)
    else:
      dilation=(2,2)

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same', dilation_rate=dilation,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def ResNett50(input_tensor=None, input_shape=(480,640,3)):

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a',strides=(1, 1)) #change attempted here
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a',d=1, strides=(1,1))  #change attempted here
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b',d=1)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c',d=1)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d',d=1)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e',d=1)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f',d=1)

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a', strides = (1,1))  #change attempted here
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    model = Model(input_tensor, x)
    weights_path = get_file(
        'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
        WEIGHTS_PATH_NO_TOP,
        cache_subdir='models',
        md5_hash='a268eb855778b3df3c7506639542a6af')
    model.load_weights(weights_path)
    return model

Using TensorFlow backend.


Actual model

In [ ]:
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D,Input,UpSampling2D, ConvLSTM2D, Lambda
import numpy as np
import keras.backend as K
from keras import optimizers

def repeat(x):
    return K.reshape(K.repeat(K.batch_flatten(x), 4), (b_s, 4, 15, 20, 512))

def repeat_shape(s):
    return (s[0], 4) + s[1:]

inp = Input(shape=(480,640,3))
def modell(inpp):

  base_model = ResNett50(input_tensor = inpp)
  #base_model = ResNet50(weights = 'imagenet', include_top = False, input_tensor = inp)
  #base_model.load_weights('tensorflow_resnet50_now.h5')
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = Conv2D(512, (3, 3), padding = 'same', activation='relu')(x)
  #x =
  y = Lambda(repeat, repeat_shape)(x)
  #y = ConvLSTM2D(512, (3, 3), padding = 'same', return_sequences = True)(y)  ##
  op_lstm = ConvLSTM2D(512, (3, 3), padding = 'same', return_sequences = False)(y)
  op_lstm = Conv2D(512, (3, 3), dilation_rate =(3,3), padding = 'same', activation='relu')(op_lstm)
  op_lstm = Conv2D(512, (3, 3), dilation_rate =(3,3), padding = 'same', activation='relu')(op_lstm)
  out = UpSampling2D(size = (32,32), interpolation = 'bilinear')(op_lstm)   #change attempted here from (32,32) to (2,2)
  out = Conv2D(1, (1, 1), padding='same', activation='relu')(out)
  return [out,out]

model = Model(inputs = inp, outputs = modell(inp))


model.summary()
opt=optimizers.RMSprop(lr=1e-4)
model.compile(optimizer=opt, loss=['kullback_leibler_divergence','mean_squared_error'],metrics=['accuracy'])
#for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)
#out =[out,out,out]
#out = UpSampling2D(size = (4,4), interpolation = 'bilinear')(out)







/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
_________________________

Don't execute this one

In [ ]:
from keras import backend as K
from keras.utils.conv_utils import convert_kernel
import tensorflow as tf
from keras.layers import Input
from keras.applications import ResNet50
ops = []
inp=Input((3,480,640))
base_model = ResNet50(weights = 'imagenet',include_top=False,input_tensor=inp)
for layer in base_model.layers:
   if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
      original_w = K.get_value(layer.W)
      converted_w = convert_kernel(original_w)
      ops.append(tf.assign(layer.W, converted_w).op)
K.get_session().run(ops)
model.save_weights('tensorflow_resnet50_now.h5')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Training part actually starts from here:

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
b_s=1
nb_imgs_train=5000
nb_imgs_val=1500
nb_epoch=20
model.load_weights('f101--74.0828.h5')
model.fit_generator(generator(), steps_per_epoch=nb_imgs_train//b_s, epochs=nb_epoch,
                    validation_data=generator(phase_gen='val'), validation_steps=nb_imgs_val//b_s,
                    callbacks=[EarlyStopping(monitor='val_loss',mode='min',patience=3)
                               ,ModelCheckpoint("f1{epoch:02d}-{val_acc:.4f}.h5", verbose=1,
                                                save_best_only=True, period =1,
                                                save_weights_only=True), reduce_lr])

OSError: ignored

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
import os
b_s=1
nb_imgs_train=5000
nb_imgs_val=2500
nb_epoch=20
#model.load_weights('/content/drive/My Drive/models/newmod/f101-0.0720.h5')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', cooldown=0, min_lr=0)
model.fit_generator(generator(), steps_per_epoch=nb_imgs_train//b_s, epochs=nb_epoch,
                    validation_data=generator(phase_gen='val'), validation_steps=nb_imgs_val//b_s,
                    callbacks=[EarlyStopping(patience=3)
                               ,ModelCheckpoint("f1{epoch:02d}-{val_loss:.4f}.h5",  verbose=1,
                                                save_best_only=False, period =1,
                                                save_weights_only=True),reduce_lr])

Epoch 1/20
5000
5000


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


10000
10000
5000/5000 [==============================] - 4890s 978ms/step - loss: 0.0070 - conv2d_9_loss: 0.0389 - conv2d_9_acc: 0.5025 - conv2d_9_acc_1: 0.5025 - val_loss: 0.0038 - val_conv2d_9_loss: 0.0220 - val_conv2d_9_acc: 0.4802 - val_conv2d_9_acc_1: 0.4802

Epoch 00001: val_loss improved from inf to 0.00376, saving model to f101-0.0038.h5
Epoch 2/20
5000/5000 [==============================] - 4693s 939ms/step - loss: -0.0026 - conv2d_9_loss: 0.0342 - conv2d_9_acc: 0.5013 - conv2d_9_acc_1: 0.5013 - val_loss: -0.0043 - val_conv2d_9_loss: 0.0419 - val_conv2d_9_acc: 0.4715 - val_conv2d_9_acc_1: 0.4715

Epoch 00002: val_loss improved from 0.00376 to -0.00425, saving model to f102--0.0043.h5
Epoch 3/20
5000/5000 [==============================] - 779s 156ms/step - loss: -0.0056 - conv2d_9_loss: 0.0332 - conv2d_9_acc: 0.5038 - conv2d_9_acc_1: 0.5038 - val_loss: 0.0064 - val_conv2d_9_loss: 0.0232 - val_conv2d_9_acc: 0.4801 - val_conv2d_9_acc_1: 0.4801

Epoch 00003: val_loss did not imp

In [ ]:
nb_imgs_test = 6
import os
#model.load_weights('/content/f102-0.0690.h5')
model.load_weights('/content/drive/My Drive/models/newmod/f102--0.0043.h5')
#model.load_weights('/content/drive/My Drive/models/newmod/f103-0.0167.h5')
load_path='/content/drive/My Drive/dataset/mini/'   #'/content/drive/My Drive/models/'
images=[f for f in os.listdir(load_path) if f.endswith(('.jpg', '.jpeg'))]#, '.png'))]
images.sort()
print(len(images))
print('images loaded.')
#imgs=preprocess_images(images[0:3])
#del images
#print(imgs.shape)
output_folder='/content/drive/My Drive/dataset/reviewoutput/'
def predict_gen(b_s,test_path):
  images = [test_path+f for f in os.listdir(test_path) if f.endswith(('.jpg', '.jpeg'))]#, '.png'))]
  images.sort()
  counterr=0
  while True:
      yield [preprocess_images(images[counterr:counterr + b_s])]
      counterr = (counterr + b_s) % len(images)

predictions = model.predict_generator(predict_gen(b_s=b_s, test_path=load_path), nb_imgs_test)[0]

for pred, name in zip(predictions, images):
    original_image = cv2.imread(load_path+name, 0)
    pred=pred.transpose(2,0,1)
    res = postprocess_predictions(pred[:][:][0], original_image.shape[0], original_image.shape[1])
    cv2.imwrite(output_folder + '%s' % name, res.astype(int))
    print(name)


6
images loaded.
postprediction generated
COCO_test2014_000000008969.jpg
postprediction generated
COCO_test2014_000000008974.jpg
postprediction generated
COCO_test2014_000000008993.jpg
postprediction generated
COCO_test2014_000000009046.jpg
postprediction generated
COCO_test2014_000000009207.jpg
postprediction generated
Copy of COCO_train2014_000000000389.jpg
